In [ ]:
# === SSC bootstrap cell (no borrar) ===
import os, sys, pathlib
BASE = "/content/drive/MyDrive/SSC"
os.environ["SSC_ROOT"] = BASE
os.environ["SSC_CACHE_DIR"] = f"{BASE}/.ssc_cache"
pathlib.Path(os.environ["SSC_CACHE_DIR"]).mkdir(parents=True, exist_ok=True)
sys.path.insert(0, BASE)

from ssc_pipeline.ssc_pipeline import run as ssc_update
ssc_update()  # refresca /content/drive/MyDrive/SSC/main.csv

import pandas as pd
df = pd.read_csv(f"{BASE}/main.csv")
print("main.csv cargado desde:", f"{BASE}/main.csv", "| Filas:", len(df))
df.head(3)


In [ ]:
import re
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define the path inside Google Drive
base_dir = "/content/drive/My Drive/Winter Quarter 2025/STA 141C - Final Project/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create the directory if it doesn't exist
os.makedirs(base_dir + "Data/", exist_ok=True)
os.makedirs(base_dir + "Data/USImpExp/", exist_ok=True)

# Define the range of years
year_range = range(2000, 2023)

# Define a dictionary to store the import/export data for each year
year_data = {}

In [ ]:
# Iterate through each year
for year in year_range:

    # Print Progress
    print(f"Scraping data for {year}:")

    # Get the HTML response from the webpage
    url = f"https://wits.worldbank.org/CountryProfile/en/Country/USA/Year/{year}/TradeFlow/EXPIMP/Partner/by-country#"
    response = requests.get(url)
    print("- Status Code:" + str(response.status_code))

    # Create the soup object for the HTML response
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the script tag containing the table data
    script_tags = soup.find_all("script", string=re.compile(r'var col\d+ ='))

    # Extract JavaScript table data
    data = {}
    columns = []
    if script_tags:
        script_content = script_tags[0].string  # Get the script content

        # Extract columns (col0, col1, etc.)
        for col_match in re.finditer(r"var (col\d+) =\s+\[([^\]]+)\];", script_content):
            col_name = col_match.group(1)  # e.g., col0, col1
            col_data = eval(col_match.group(2))  # Convert the array string to a Python list
            data[col_name] = col_data
            columns.append(col_name)

        # Extract column headers
        headers_match = re.search(r"columns: \[\{ text:'([^']+)", script_content)
        headers = []
        if headers_match:
            headers = [header.group(1) for header in re.finditer(r"text:'([^']+)", script_content)]

        # If headers are empty, assign generic names
        if not headers: headers = [f"Column {i}" for i in range(len(columns))]

        # Convert to DataFrame
        df = pd.DataFrame(data).rename(columns={f"col{i}": headers[i] for i in range(len(headers))})

        # Save the dataframe to the dictionary
        year_data[year] = df

        # Save the data to files
        df.to_csv(base_dir + f"Data/USImpExp/{year}.csv", index = False)

    # Avoid server overloading
    time.sleep(0.05)

Scraping data for 2000:
- Status Code:200
Scraping data for 2001:
- Status Code:200
Scraping data for 2002:
- Status Code:200
Scraping data for 2003:
- Status Code:200
Scraping data for 2004:
- Status Code:200
Scraping data for 2005:
- Status Code:200
Scraping data for 2006:
- Status Code:200
Scraping data for 2007:
- Status Code:200
Scraping data for 2008:
- Status Code:200
Scraping data for 2009:
- Status Code:200
Scraping data for 2010:
- Status Code:200
Scraping data for 2011:
- Status Code:200
Scraping data for 2012:
- Status Code:200
Scraping data for 2013:
- Status Code:200
Scraping data for 2014:
- Status Code:200
Scraping data for 2015:
- Status Code:200
Scraping data for 2016:
- Status Code:200
Scraping data for 2017:
- Status Code:200
Scraping data for 2018:
- Status Code:200
Scraping data for 2019:
- Status Code:200
Scraping data for 2020:
- Status Code:200
Scraping data for 2021:
- Status Code:200
Scraping data for 2022:
- Status Code:200


In [ ]:
year_data[2022].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 35 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Partner Name                             223 non-null    object
 1   No Of exported HS6 digit Products        223 non-null    object
 2   No Of imported HS6 digit Products        223 non-null    object
 3   Export Share in Total Products (%)       223 non-null    object
 4   Import Share in Total Products (%)       223 non-null    object
 5   Trade Balance (US$ Thousand)             223 non-null    object
 6   Export (US$ Thousand)                    223 non-null    object
 7   Import (US$ Thousand)                    223 non-null    object
 8   Import Partner Share (%)                 223 non-null    object
 9   Export Partner Share (%)                 223 non-null    object
 10  AHS Simple Average (%)                   223 non-null    objec

# Breakdown of Each Column in Your DataFrame

## **Trade Partner Information**
1. **Partner Name** – The name of the country or region that trades with the U.S.
2. **No Of exported HS6 digit Products** – The number of different products (at the HS6 level) the U.S. exports to that partner.
3. **No Of imported HS6 digit Products** – The number of different products (at the HS6 level) the U.S. imports from that partner.

## **Trade Value and Share**
4. **Export Share in Total Products (%)** – The percentage of total exported products that go to this partner.
5. **Import Share in Total Products (%)** – The percentage of total imported products that come from this partner.
6. **Trade Balance (US$ Thousand)** – The difference between exports and imports with this partner (positive = trade surplus, negative = trade deficit).
7. **Export (US\$ Thousand)** – The total value of exports from the U.S. to this partner.
8. **Import (US\$ Thousand)** – The total value of imports from this partner to the U.S.
9. **Import Partner Share (%)** – The percentage of the U.S.’s total imports that come from this partner.
10. **Export Partner Share (%)** – The percentage of the U.S.’s total exports that go to this partner.

## **Applied Tariff Rates (AHS - Applied Tariff Schedule)**
These columns represent tariffs actually applied to imports from this partner under trade agreements and policies:

11. **AHS Simple Average (%)** – The average tariff rate across all tariff lines, without considering trade volume.  
12. **AHS Weighted Average (%)** – The average tariff rate, weighted by the trade value of each product.  
13. **AHS Total Tariff Lines** – The total number of tariff lines applied to this partner’s imports.  
14. **AHS Dutiable Tariff Lines Share (%)** – The percentage of tariff lines that have duties.  
15. **AHS Duty Free Tariff Lines Share (%)** – The percentage of tariff lines that allow duty-free imports.  
16. **AHS Specific Tariff Lines Share (%)** – The percentage of tariff lines that have a specific (rather than percentage-based) duty.  
17. **AHS AVE Tariff Lines Share (%)** – The percentage of tariff lines with an ad valorem equivalent (AVE) tariff.  
18. **AHS MaxRate (%)** – The highest applied tariff rate.  
19. **AHS MinRate (%)** – The lowest applied tariff rate.  
20. **AHS SpecificDuty Imports (US\$ Thousand)** – The value of imports subject to specific (non-ad valorem) duties.  
21. **AHS Dutiable Imports (US\$ Thousand)** – The value of imports that are subject to any duty.  
22. **AHS Duty Free Imports (US\$ Thousand)** – The value of imports that are duty-free.  

## **Most Favored Nation (MFN) Tariff Rates**
These columns represent the standard tariffs applied to WTO members that do not have special trade agreements with the U.S.:

23. **MFN Simple Average (%)** – The average tariff rate across all MFN tariff lines.  
24. **MFN Weighted Average (%)** – The weighted average tariff rate under MFN rules.  
25. **MFN Total Tariff Lines** – The total number of tariff lines under MFN treatment.  
26. **MFN Dutiable Tariff Lines Share (%)** – The percentage of MFN tariff lines that have duties.  
27. **MFN Duty Free Tariff Lines Share (%)** – The percentage of MFN tariff lines that are duty-free.  
28. **MFN Specific Tariff Lines Share (%)** – The percentage of MFN tariff lines with specific (non-ad valorem) duties.  
29. **MFN AVE Tariff Lines Share (%)** – The percentage of MFN tariff lines with ad valorem equivalent tariffs.  
30. **MFN MaxRate (%)** – The highest tariff rate under MFN.  
31. **MFN MinRate (%)** – The lowest tariff rate under MFN.  
32. **MFN SpecificDuty Imports (US\$ Thousand)** – The value of imports subject to specific duties under MFN.  
33. **MFN Dutiable Imports (US\$ Thousand)** – The value of MFN imports subject to any duty.  
34. **MFN Duty Free Imports (US\$ Thousand)** – The value of MFN imports that are duty-free.  

## **Trade Agreements**
35. **No Of Tariff Agreement** – The number of trade agreements that affect tariffs for this partner.

---

## **Key Takeaways**
- The **AHS** columns reflect the actual tariffs applied due to trade agreements.  
- The **MFN** columns reflect the default tariffs under WTO rules.  
- The **Trade Balance** column helps identify whether the U.S. has a surplus or deficit with each partner.

In [ ]:
countries_dir = "/content/drive/My Drive/Winter Quarter 2025/STA 141C - Final Project/Data/ByCountry/"
years_dir = "/content/drive/My Drive/Winter Quarter 2025/STA 141C - Final Project/Data/ByYear/"
products_dir = "/content/drive/My Drive/Winter Quarter 2025/STA 141C - Final Project/Data/ByProduct/"
main_df = pd.DataFrame()

countries = {
    'China': 'CHN',
    'Canada': 'CAN',
    'Germany': 'DEU',
    'Japan': 'JPN',
    'Mexico': 'MEX'
}

# Create the directory if it doesn't exist
os.makedirs(countries_dir, exist_ok=True)
for country in list(countries.keys()): os.makedirs(countries_dir + country, exist_ok = True)
os.makedirs(years_dir, exist_ok=True)
for year in year_range: os.makedirs(years_dir + str(year), exist_ok = True)
os.makedirs(products_dir, exist_ok=True)

In [ ]:
for country in list(countries.keys()):

  # Loop through the years from 1991 to 2022
  for year in year_range:

    # Print Progress
    print(f"Scraping Import/Export Data for {country} {year}:")

    # Get the response for the specific year
    url = f"https://wits.worldbank.org/CountryProfile/en/Country/USA/Year/{year}/TradeFlow/EXPIMP/Partner/{countries[country]}/Product/all-groups"
    response = requests.get(url)
    print(f"- Response: {response.status_code}")

    # Extract the script tag containing the table data
    soup = BeautifulSoup(response.text, "html.parser")
    script_tags = soup.find_all('script', string=re.compile(r'var col\d+ ='))

    # Parse the JavaScript variables
    if not script_tags: continue
    data, columns = {}, []

    script_content = script_tags[0].string  # Get the script content
    for col_match in re.finditer(r"var (col\d+) =\s+\[([^\]]+)\];", script_content):
        col_name = col_match.group(1)  # Column name (e.g., col0, col1)
        col_data = eval(col_match.group(2))  # Convert the array string to a Python list
        data[col_name] = col_data
        columns.append(col_name)

    # Extract column headers from another script variable, if available
    headers_match = re.search(r"columns: \[\{ text:'([^']+)", script_content)
    headers = []
    if headers_match:
        headers = [header.group(1) for header in re.finditer(r"text:'([^']+)", script_content)]

    # If headers are empty, assign default headers
    if not headers: headers = [f"Column {i}" for i in range(len(columns))]

    # Create DataFrame for the current year
    df = pd.DataFrame(data).rename(columns={f"col{i}": headers[i] for i in range(len(headers))})

    # Save to drive
    df.to_csv(f"{countries_dir}{country}/{year}.csv", index = False)
    df.to_csv(f"{years_dir}{year}/{country}.csv", index = False)

    # Append to the main dataframe
    df['Country'], df['Year'] = country, year
    main_df = pd.concat([main_df, df], ignore_index = True)

    # Limit Requests to ~20 per second
    time.sleep(0.05)

Scraping Import/Export Data for China 2000:
- Response: 200
Scraping Import/Export Data for China 2001:
- Response: 200
Scraping Import/Export Data for China 2002:
- Response: 200
Scraping Import/Export Data for China 2003:
- Response: 200
Scraping Import/Export Data for China 2004:
- Response: 200
Scraping Import/Export Data for China 2005:
- Response: 200
Scraping Import/Export Data for China 2006:
- Response: 200
Scraping Import/Export Data for China 2007:
- Response: 200
Scraping Import/Export Data for China 2008:
- Response: 200
Scraping Import/Export Data for China 2009:
- Response: 200
Scraping Import/Export Data for China 2010:
- Response: 200
Scraping Import/Export Data for China 2011:
- Response: 200
Scraping Import/Export Data for China 2012:
- Response: 200
Scraping Import/Export Data for China 2013:
- Response: 200
Scraping Import/Export Data for China 2014:
- Response: 200
Scraping Import/Export Data for China 2015:
- Response: 200
Scraping Import/Export Data for China 20

In [ ]:
main_df

,Product Group,Export (US$ Thousand),Import (US$ Thousand),Export Product Share (%),Import Product Share (%),Revealed comparative advantage,World Growth (%),Country Growth (%),AHS Simple Average (%),AHS Weighted Average (%),...,MFN Duty Free Tariff Lines Share (%),MFN Specific Tariff Lines Share (%),MFN AVE Tariff Lines Share (%),MFN MaxRate (%),MFN MinRate (%),MFN SpecificDuty Imports (US$ Thousand),MFN Dutiable Imports (US$ Thousand),MFN Duty Free Imports (US$ Thousand),Country,Year
0,All Products,16184679.30,100012902.71,100.00,100.00,1.00,16.55,7.15,4.24,4.11,...,30.50,2.04,8.06,350.00,0.00,1501148.82,62928834.48,43538381.04,China,2000
1,Capital goods,8304745.38,26136596.70,51.31,26.13,1.27,15.75,6.70,1.48,0.79,...,54.80,0.27,0.60,25.00,0.00,19163.62,10335499.23,17008951.99,China,2000
2,Consumer goods,1325146.86,64141852.76,8.19,64.13,0.73,22.41,10.64,5.29,5.55,...,22.77,3.81,9.40,247.47,0.00,943810.83,47858993.09,21810579.77,China,2000
3,Intermediate goods,3233539.85,7023412.27,19.98,7.02,0.74,11.13,-0.61,5.21,3.51,...,22.69,1.27,7.95,350.00,0.00,95138.13,3909747.78,3815256.07,China,2000
4,Raw materials,2483821.28,1574956.45,15.35,1.57,1.04,42.98,30.99,1.91,1.25,...,54.35,0.99,22.33,350.00,0.00,443036.24,824594.38,903593.21,China,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2410,Stone and Glass,2682233.32,9364003.11,0.83,2.04,0.87,7.56,5.90,0.00,0.00,...,33.72,0.00,1.45,38.00,0.00,0.00,7543711.97,1452397.26,Mexico,2022
2411,Textiles and Clothing,6424058.89,5680268.04,1.98,1.24,0.60,10.06,3.87,0.00,0.00,...,12.56,0.00,4.11,32.00,0.00,0.00,5331392.26,144592.42,Mexico,2022
2412,Transportation,24459899.41,113901777.81,7.54,24.81,1.02,11.06,8.73,0.00,0.00,...,46.08,0.00,0.00,25.00,0.00,0.00,93808285.22,3814814.58,Mexico,2022
2413,Vegetable,14308950.99,21043859.89,4.41,4.58,1.53,7.20,4.33,0.00,0.00,...,23.76,0.00,41.68,131.80,0.00,0.00,19054722.73,1486948.45,Mexico,2022


In [ ]:
# Iterate over unique product groups
for product_name in main_df['Product Group'].unique():

  # Create a subdirectory for each product group
  product_subdir = os.path.join(products_dir, str(product_name))
  os.makedirs(product_subdir, exist_ok=True)

  # Filter the main DataFrame for the current product group
  product_df = main_df[main_df['Product Group'] == product_name]

  # Iterate over unique countries within the current product group
  for country_name in product_df['Country'].unique():
    # Filter the product DataFrame for the current country
    country_df = product_df[product_df['Country'] == country_name]

    # Remove 'Country' and 'Product Group' columns
    country_df = country_df.drop(columns=['Country', 'Product Group'])

    # Define the output file path
    output_file = os.path.join(product_subdir, f"{country_name}.csv")

    # Print Progress:
    print(f"Saving {output_file}...")

    # Save the data to a CSV file
    country_df.to_csv(output_file, index=False)


Saving /content/drive/My Drive/Winter Quarter 2025/STA 141C - Final Project/Data/ByProduct/All Products/China.csv...
Saving /content/drive/My Drive/Winter Quarter 2025/STA 141C - Final Project/Data/ByProduct/All Products/Canada.csv...
Saving /content/drive/My Drive/Winter Quarter 2025/STA 141C - Final Project/Data/ByProduct/All Products/Germany.csv...
Saving /content/drive/My Drive/Winter Quarter 2025/STA 141C - Final Project/Data/ByProduct/All Products/Japan.csv...
Saving /content/drive/My Drive/Winter Quarter 2025/STA 141C - Final Project/Data/ByProduct/All Products/Mexico.csv...
Saving /content/drive/My Drive/Winter Quarter 2025/STA 141C - Final Project/Data/ByProduct/Capital goods/China.csv...
Saving /content/drive/My Drive/Winter Quarter 2025/STA 141C - Final Project/Data/ByProduct/Capital goods/Canada.csv...
Saving /content/drive/My Drive/Winter Quarter 2025/STA 141C - Final Project/Data/ByProduct/Capital goods/Germany.csv...
Saving /content/drive/My Drive/Winter Quarter 2025/STA

In [ ]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2415 entries, 0 to 2414
Data columns (total 34 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Product Group                            2415 non-null   object
 1   Export (US$ Thousand)                    2415 non-null   object
 2   Import (US$ Thousand)                    2415 non-null   object
 3   Export Product Share (%)                 2415 non-null   object
 4   Import Product Share (%)                 2415 non-null   object
 5   Revealed comparative advantage           2415 non-null   object
 6   World Growth (%)                         2415 non-null   object
 7   Country Growth (%)                       2415 non-null   object
 8   AHS Simple Average (%)                   2415 non-null   object
 9   AHS Weighted Average (%)                 2415 non-null   object
 10  AHS Total Tariff Lines                   2415 non-null   obj

# Breakdown of Each Column in Your DataFrame

## **Product-Level Trade Data**
1. **Product Group** – The category of goods being traded (e.g., electronics, machinery, textiles).  
2. **Export (US\$ Thousand)** – The total value of exports from the U.S. for this product group.  
3. **Import (US\$ Thousand)** – The total value of imports to the U.S. for this product group.  
4. **Export Product Share (%)** – The percentage of this product group relative to total U.S. exports.  
5. **Import Product Share (%)** – The percentage of this product group relative to total U.S. imports.  
6. **Revealed Comparative Advantage** – A measure of how competitive the U.S. is in exporting this product compared to global trade patterns.  

## **Trade Growth Rates**  
7. **World Growth (%)** – The global growth rate of trade for this product group.  
8. **Country Growth (%)** – The growth rate of U.S. trade (exports/imports) for this product group.  

## **Applied Tariff Rates (AHS - Applied Tariff Schedule)**  
These columns represent tariffs actually applied to imports from trade partners under agreements and policies:  

9. **AHS Simple Average (%)** – The unweighted average tariff rate across all tariff lines for this product.  
10. **AHS Weighted Average (%)** – The trade-weighted average tariff rate for this product.  
11. **AHS Total Tariff Lines** – The total number of tariff lines applicable to this product.  
12. **AHS Dutiable Tariff Lines Share (%)** – The percentage of tariff lines that have duties for this product.  
13. **AHS Duty Free Tariff Lines Share (%)** – The percentage of tariff lines that allow duty-free imports.  
14. **AHS Specific Tariff Lines Share (%)** – The percentage of tariff lines with specific (non-ad valorem) duties.  
15. **AHS AVE Tariff Lines Share (%)** – The percentage of tariff lines with ad valorem equivalent (AVE) tariffs.  
16. **AHS MaxRate (%)** – The highest applied tariff rate for this product.  
17. **AHS MinRate (%)** – The lowest applied tariff rate for this product.  
18. **AHS SpecificDuty Imports (US\$ Thousand)** – The value of imports subject to specific duties (non-ad valorem).  
19. **AHS Dutiable Imports (U\$ Thousand)** – The value of imports that are subject to any duty.  
20. **AHS Duty Free Imports (US\$ Thousand)** – The value of imports that are duty-free.  

## **Most Favored Nation (MFN) Tariff Rates**  
These columns represent the standard tariffs applied to WTO members without special trade agreements:  

21. **MFN Simple Average (%)** – The unweighted average MFN tariff rate for this product.  
22. **MFN Weighted Average (%)** – The trade-weighted average MFN tariff rate.  
23. **MFN Total Tariff Lines** – The total number of tariff lines under MFN treatment.  
24. **MFN Dutiable Tariff Lines Share (%)** – The percentage of MFN tariff lines that have duties.  
25. **MFN Duty Free Tariff Lines Share (%)** – The percentage of MFN tariff lines that are duty-free.  
26. **MFN Specific Tariff Lines Share (%)** – The percentage of MFN tariff lines with specific (non-ad valorem) duties.  
27. **MFN AVE Tariff Lines Share (%)** – The percentage of MFN tariff lines with ad valorem equivalent tariffs.  
28. **MFN MaxRate (%)** – The highest MFN tariff rate for this product.  
29. **MFN MinRate (%)** – The lowest MFN tariff rate for this product.  
30. **MFN SpecificDuty Imports (U\$ Thousand)** – The value of imports subject to specific duties under MFN rules.  
31. **MFN Dutiable Imports (US\$ Thousand)** – The value of MFN imports subject to any duty.  
32. **MFN Duty Free Imports (US\$ Thousand)** – The value of MFN imports that are duty-free.  

## **Country and Year Information**  
33. **Country** – The trading partner country for which the data applies.  
34. **Year** – The year of the trade data.  

In [ ]:
main_df.to_csv(base_dir + "Data/main.csv", index = False)